In [2]:
# Install required libraries
!pip install pandas pyodbc

In [8]:
import pandas as pd
import pyodbc
import os


In [10]:
# Define dataset path
dataset_path = r"C:\Users\neloya\OneDrive - Douglas College\GitHub\UBC\Winter 2025\FRE521D\FRE521D-Winter2025_UBC\Datasets\AirQualityUCI.csv"

# Check if the file exists before proceeding
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset not found at {dataset_path}")


In [12]:
# Establish connection to SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'  # Change this if using a different server
    'DATABASE=FRE521D;'  # Use your actual database
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()


In [14]:
# Load CSV file
df = pd.read_csv(dataset_path, delimiter=";")

# Display first few rows
df.head()


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [18]:
# Remove unnamed columns and completely empty columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Remove "Unnamed" columns
df = df.dropna(axis=1, how='all')  # Remove columns where all values are NaN

# Remove completely empty rows
df = df.dropna(axis=0, how='all')  # Remove rows where all values are NaN

# Reset index after dropping rows
df = df.reset_index(drop=True)

# Display the cleaned dataset
df


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,04/04/2005,10.00.00,"3,1",1314.0,-200.0,"13,5",1101.0,472.0,539.0,190.0,1374.0,1729.0,"21,9","29,3","0,7568"
9353,04/04/2005,11.00.00,"2,4",1163.0,-200.0,"11,4",1027.0,353.0,604.0,179.0,1264.0,1269.0,"24,3","23,7","0,7119"
9354,04/04/2005,12.00.00,"2,4",1142.0,-200.0,"12,4",1063.0,293.0,603.0,175.0,1241.0,1092.0,"26,9","18,3","0,6406"
9355,04/04/2005,13.00.00,"2,1",1003.0,-200.0,"9,5",961.0,235.0,702.0,156.0,1041.0,770.0,"28,3","13,5","0,5139"


In [20]:
# Function to map Pandas data types to SQL Server data types
def map_dtype(dtype):
    if "int" in str(dtype):
        return "INT"
    elif "float" in str(dtype):
        return "FLOAT"
    else:
        return "VARCHAR(255)"

# Generate CREATE TABLE query dynamically
table_name = "AirQuality"
columns = ", ".join([f"[{col}] {map_dtype(dtype)}" for col, dtype in zip(df.columns, df.dtypes)])

create_table_query = f"CREATE TABLE {table_name} ({columns});"

# Execute the CREATE TABLE statement
cursor.execute(create_table_query)
conn.commit()

print(f"Table {table_name} created successfully!")


Table AirQuality created successfully!


In [22]:
# Replace NaN values with None to prevent SQL errors
df = df.where(pd.notnull(df), None)

# Insert data into SQL table
for index, row in df.iterrows():
    placeholders = ", ".join(["?"] * len(row))
    insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
    
    cursor.execute(insert_query, tuple(row))

conn.commit()

print(f"Data inserted into {table_name} successfully!")


Data inserted into AirQuality successfully!


In [24]:
# Query the table to check inserted data
query = f"SELECT TOP 5 * FROM {table_name};"
df_sql = pd.read_sql(query, conn)
df_sql


C:\Users\neloya\AppData\Local\Temp\ipykernel_23260\3596644092.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, conn)


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"
